# Sensor - Wheel Encoders

In this notebook we will learn how to use the wheel encoder component to retrieve wheel rotation readings from a Duckiebot.

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import cv2
import matplotlib.pyplot as plt


# TODO: change this to the name of your Duckiebot
VEHICLE_NAME: str = "db21j3"

In [ ]:
# Sensor - Wheel Encoders - Read ticks from single wheel

from typing import Optional
from duckietown.components.duckiebot import WheelEncoderDriverComponent

# define component
left_wheel_encoder: WheelEncoderDriverComponent = WheelEncoderDriverComponent(vehicle_name=VEHICLE_NAME, side="left")
left_wheel_encoder.start()

# wait for next reading
ticks: int = left_wheel_encoder.out_ticks.get()

# print number of ticks (since the robot turned ON)
print(f"Number of ticks: {ticks}\nResolution: {left_wheel_encoder.resolution} ticks/2π")

# stop component
left_wheel_encoder.stop()

In [ ]:
# Sensor - Time-of-Flight - Continuous reading from both wheel

from typing import Optional, Tuple
from functools import partial

from duckietown.components.duckiebot import WheelEncoderDriverComponent 
from duckietown.components.utilities import SynchronizerComponent
from duckietown.components.rendering import MarkdownRendererComponent


WHEEL_RADIUS: float = 0.0318
scale_meters_per_pixel = 1 / 400  # 1 meter = 400 pixels

def convert_meters_to_pixels(distance_meters, scale_meters_per_pixel):
    """
    Convert distance from meters to pixels.

    :param distance_meters: Distance traveled in meters.
    :param scale_meters_per_pixel: The scale of the map in meters per pixel.
    :return: Equivalent distance in pixels.
    """
    pixel_distance = distance_meters / scale_meters_per_pixel
    return pixel_distance

# define format function
def format(left_wheel_encoder: WheelEncoderDriverComponent, right_wheel_encoder: WheelEncoderDriverComponent, ticks: Tuple[int, int]) -> str:
    left_ticks, right_ticks = ticks
    left_revols, right_revols = left_ticks / left_wheel_encoder.resolution, right_ticks / right_wheel_encoder.resolution
    left_rads, right_rads = left_revols * 2 * np.pi, right_revols * 2 * np.pi
    left_odom, right_odom = 2 * np.pi * WHEEL_RADIUS * left_revols, 2 * np.pi * WHEEL_RADIUS * right_revols
    left_pixels, right_pixels = convert_meters_to_pixels(left_odom, scale_meters_per_pixel), convert_meters_to_pixels(right_odom, scale_meters_per_pixel)
    text: str = f"""
| Wheel              | # Ticks       | Radians              | # Revolutions      | Odometer           | Pixels             |
| :----------------- | :-----------: | -------------------: | -----------------: | -----------------: | -----------------: |
| Left               | {left_ticks}  | {left_rads:.2f} rad  | {left_revols:.2f}  | {left_odom:.2f} m  | {left_pixels} px   |
| Right              | {right_ticks} | {right_rads:.2f} rad | {right_revols:.2f} | {right_odom:.2f} m | {right_pixels} px  |
"""
    return text

# define components
left_wheel_encoder: WheelEncoderDriverComponent = WheelEncoderDriverComponent(vehicle_name=VEHICLE_NAME, side="left")
right_wheel_encoder: WheelEncoderDriverComponent = WheelEncoderDriverComponent(vehicle_name=VEHICLE_NAME, side="right")
synchronizer: SynchronizerComponent = SynchronizerComponent((left_wheel_encoder.out_ticks, right_wheel_encoder.out_ticks))
renderer: MarkdownRendererComponent = MarkdownRendererComponent(formatter=partial(format, left_wheel_encoder, right_wheel_encoder))

# connect components
renderer.in_data.wants(synchronizer.out_data)

# start components
left_wheel_encoder.start()
right_wheel_encoder.start()
renderer.start()
synchronizer.start()

# wait until the cell is stopped
renderer.join()

# stop components
left_wheel_encoder.stop()
right_wheel_encoder.stop()
renderer.stop()
synchronizer.stop()
import matplotlib.pyplot as plt

# Create an empty map
map_width = 1300
map_height = 1900
map = np.zeros((map_height, map_width))

# Set the starting position
start_x = 600
start_y = 800

# Initialize the current position
current_x = start_x
current_y = start_y

# Draw the robot's path
for left_pixel, right_pixel in zip(left_pixels, right_pixels):
    # Calculate the change in x and y coordinates
    delta_x = (left_pixel + right_pixel) / 2
    delta_y = (right_pixel - left_pixel) / 2

    # Update the current position
    current_x += delta_x
    current_y += delta_y

    # Draw a line from the previous position to the current position
    plt.plot([prev_x, current_x], [prev_y, current_y], color='red')

    # Update the previous position
    prev_x = current_x
    prev_y = current_y

# Show the map
plt.imshow(map, cmap='gray')
plt.show()



